In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
import time
import warnings
import json
from pyspark.sql import Window
from pyspark.sql.functions import lag, col
import mysql.connector as database
import time

# from mock.tasks import adiciona_carro}
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder \
           .appName('SparkByExamples') \
           .config("spark.jars", "/usr/share/java/mariadb-java-client.jar") \
           .getOrCreate()
sql = SQLContext(ss)

warnings.simplefilter(action='ignore', category=FutureWarning)

session.execute("USE simulacao")

params = json.load(open('./mock/parametros.json'))

connection = database.connect(
    host="host.docker.internal",
    port=3306,
    user="root",
    password="secret"
)

cursor = connection.cursor()
cursor.execute("USE dashboard;")

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# list(session.execute("SELECT MAX(tempo_da_simulacao) FROM simulacao"))

In [3]:
# list(session.execute("SELECT COUNT(*) FROM simulacao"))[0][0]

In [4]:
start_time = time.time()
total = 0
for i in range(10):
    start_time2 = time.time()
    a, b = 1000*i, 1000*i + 100
    query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
    r = list(session.execute(query))
    if r != []:
        df = ss.createDataFrame(r)
        
        p = [[key]+list(params[key].values()) for key in params.keys()]
        p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))
        
        Velocidades_Maximas = p.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
        Aceleracoes_Maximas = p.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
        Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")
        
        collision_tolerance = 1
        collision_tolerance_quad = 0.5
        windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")
        
        df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
        df = df.withColumn("prev_tempo_da_simulacao", lag("tempo_da_simulacao", 1).over(windowSpec))
        df = df.withColumn("vel_y", (col("pos_y") - col("prev_pos_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
        df = df.withColumn("prev_vel_y", lag("vel_y", 1).over(windowSpec))
        df = df.withColumn("acel_y", (col("vel_y") - col("prev_vel_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
        
        df = df.withColumn("posicao_prevista", col("pos_y") + col("vel_y") * (collision_tolerance) + col("acel_y") * collision_tolerance_quad)
        
        window_spec_rf = Window.partitionBy("rodovia", "pos_x").orderBy('pos_y')
        lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec_rf)
        lead_column = lead(col("posicao_prevista")).over(window_spec_rf) - col("posicao_prevista")
        
        # Add the lag column to the DataFrame
        df = df.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lag(col("pos_x")).over(window_spec_rf)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lead(col("pos_x")).over(window_spec_rf))), 1).otherwise(0))
        
        df = df.join(Velocidades_Maximas,on='rodovia',how='left')
        df = df.join(Aceleracoes_Maximas,on='rodovia',how='left')
        
        df = df.withColumn('acima_vel',F.abs(col('vel_y'))>F.abs(col('VelocidadeMaxima')))
        df = df.withColumn('acima_acel',F.abs(col('acel_y'))>F.abs(col('AceleracaoMaxima')))
        
        df = df.withColumn("troca_faixa", col("pos_x") != lag("pos_x", 1).over(windowSpec))
        
        # contador de trocas
        
        df = df.withColumn('multado',((F.col('acima_vel') == 1) & (lag('acima_vel').over(windowSpec) == 0)))
        
        windowSpec = Window.partitionBy("placa",'rodovia').orderBy('tempo_da_simulacao')
        
        df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
        df = df.withColumn("on_road", (((col("pos_y") > 0) & (col('pos_y') < 800)) & (col("prev_pos_y") > 0)).cast("int"))
        df = df.withColumn('time_on_road',F.lit(0))
    
        data = df.select('placa','pos_x','pos_y','acel_y','vel_y', 'rodovia', 'tempo_da_simulacao',F.col('multado').cast('int'),'Risco_Colisão',F.col('troca_faixa'))#,'acima_vel')
        datap = data.toPandas()
    
        query = f""" INSERT IGNORE INTO carros (placa, pos_x, pos_y, aceleracao, velocidade, rodovia, horario_registro, multas, risco_colisao, direcao_perigosa)
    
                    VALUES {','.join([str(i) for i in list(datap.to_records(index=False))])};
                    
                    """.replace("None", "NULL").replace("\n", "").replace("nan", "NULL")
    
        cursor.execute(query)
        connection.commit()

        from pyspark.sql import functions as F, Window

        # Obter o valor máximo da coluna tempo_da_simulacao
        df2 = df.select('rodovia', 'tempo_da_simulacao')
        max_tempo = df2.select(F.max("tempo_da_simulacao")).collect()[0][0]
        
        # Filtrar o dataframe pelo valor máximo
        df2 = df2.filter(df2.tempo_da_simulacao == max_tempo)
        
        # Aplicar um unique no dataframe df2
        df2 = df2.distinct()
        
        df2.show()
        
        max_tempo = df2.select(F.max("tempo_da_simulacao")).collect()[0][0]
        df2 = df2.filter(df2.tempo_da_simulacao == max_tempo)
        
        # Criar uma janela de tamanho 1 sobre a coluna rodovia
        w = Window.partitionBy(F.col("rodovia")).orderBy(F.col("rodovia")).rangeBetween(0, 0)
        
        # Contar as placas distintas por rodovia
        unique_placas = df.filter(df.tempo_da_simulacao == max_tempo_da_simulacao)\
                          .withColumn("placa_distinct", F.size(F.collect_set("placa").over(w)))\
                          .select("rodovia", "placa_distinct")\
                          .distinct()
        
        # Contar as placas distintas por rodovia que tiveram colisão
        unique_colisao = df.filter((df.tempo_da_simulacao == max_tempo_da_simulacao) & (df.vel_y == 0))\
                          .withColumn("placa_distinct_colissao", F.size(F.collect_set("placa").over(w)))\
                          .select("rodovia", "placa_distinct_colissao")\
                          .distinct()
        
        df2 = df2.join(unique_placas,on='rodovia',how='inner')
        df2 = df2.join(unique_colisao,on='rodovia',how='inner')
        
        df2

    
    
    end_time = time.time()
    elapsed_time = end_time - start_time2
    total = total + elapsed_time
    print(f"For iterarion {i} - Elapsed time: {elapsed_time} seconds")

    cursor.execute("SELECT COUNT(*) FROM carros;")
    print("Número de linhas no DB:", cursor.fetchall())

print()
print(f"Total time for {i+1} iterations: {total}")

For iterarion 0 - Elapsed time: 9.729284524917603 seconds
Número de linhas no DB: [(3255,)]
For iterarion 1 - Elapsed time: 2.9378111362457275 seconds
Número de linhas no DB: [(4089,)]
For iterarion 2 - Elapsed time: 2.1552937030792236 seconds
Número de linhas no DB: [(4197,)]
For iterarion 3 - Elapsed time: 0.034596920013427734 seconds
Número de linhas no DB: [(4197,)]
For iterarion 4 - Elapsed time: 0.04046320915222168 seconds
Número de linhas no DB: [(4197,)]
For iterarion 5 - Elapsed time: 0.03837728500366211 seconds
Número de linhas no DB: [(4197,)]
For iterarion 6 - Elapsed time: 0.017972946166992188 seconds
Número de linhas no DB: [(4197,)]
For iterarion 7 - Elapsed time: 0.016458988189697266 seconds
Número de linhas no DB: [(4197,)]
For iterarion 8 - Elapsed time: 0.01586318016052246 seconds
Número de linhas no DB: [(4197,)]
For iterarion 9 - Elapsed time: 0.025409936904907227 seconds
Número de linhas no DB: [(4197,)]

Total time for 10 iterations: 15.011531829833984


In [11]:
df.toPandas()

,rodovia,tempo_da_simulacao,placa,pos_x,pos_y,prev_pos_y,prev_tempo_da_simulacao,vel_y,prev_vel_y,acel_y,posicao_prevista,Risco_Colisão,VelocidadeMaxima,AceleracaoMaxima,acima_vel,acima_acel,troca_faixa,multado,on_road,time_on_road
0,BR-135,2030,ARG3L45,745.0,864.0,NaN,NaN,NaN,NaN,NaN,NaN,0,12,0.16,None,None,None,None,0.0,0
1,BR-135,2031,ARG3L45,745.0,867.0,864.0,2030.0,3.0,NaN,NaN,NaN,0,12,0.16,False,None,False,False,0.0,0
2,BR-135,2033,ARG3L45,745.0,858.0,867.0,2031.0,-4.5,3.0,-3.75,851.625,1,12,0.16,False,True,False,False,0.0,0
3,BR-135,2034,ARG3L45,745.0,855.0,858.0,2033.0,-3.0,-4.5,1.50,852.750,1,12,0.16,False,True,False,False,0.0,0
4,BR-135,2036,ARG3L45,745.0,846.0,855.0,2034.0,-4.5,-3.0,-0.75,841.125,1,12,0.16,False,True,False,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,BR-116,2094,VEN4E56,745.0,860.0,856.0,2093.0,4.0,-2.0,6.00,867.000,1,9,0.12,False,True,False,False,0.0,0
681,BR-116,2095,VEN4E56,745.0,854.0,860.0,2094.0,-6.0,4.0,-10.00,843.000,1,9,0.12,False,True,False,False,0.0,0
682,BR-116,2096,VEN4E56,745.0,848.0,854.0,2095.0,-6.0,-6.0,0.00,842.000,0,9,0.12,False,False,False,False,0.0,0
683,BR-116,2097,VEN4E56,745.0,850.0,848.0,2096.0,2.0,-6.0,8.00,856.000,1,9,0.12,False,True,False,False,0.0,0


In [6]:
cursor.execute("SELECT COUNT(*) FROM carros;")

In [7]:
cursor.fetchall()

[(4197,)]

In [8]:
cursor.execute("SELECT * FROM carros;")

In [10]:
# for j in cursor.fetchall():
#     print(j)

In [12]:
df.toPandas()

,rodovia,tempo_da_simulacao,placa,pos_x,pos_y,prev_pos_y,prev_tempo_da_simulacao,vel_y,prev_vel_y,acel_y,posicao_prevista,Risco_Colisão,VelocidadeMaxima,AceleracaoMaxima,acima_vel,acima_acel,troca_faixa,multado,on_road,time_on_road
0,BR-135,2030,ARG3L45,745.0,864.0,NaN,NaN,NaN,NaN,NaN,NaN,0,12,0.16,None,None,None,None,0.0,0
1,BR-135,2031,ARG3L45,745.0,867.0,864.0,2030.0,3.0,NaN,NaN,NaN,0,12,0.16,False,None,False,False,0.0,0
2,BR-135,2033,ARG3L45,745.0,858.0,867.0,2031.0,-4.5,3.0,-3.75,851.625,1,12,0.16,False,True,False,False,0.0,0
3,BR-135,2034,ARG3L45,745.0,855.0,858.0,2033.0,-3.0,-4.5,1.50,852.750,1,12,0.16,False,True,False,False,0.0,0
4,BR-135,2036,ARG3L45,745.0,846.0,855.0,2034.0,-4.5,-3.0,-0.75,841.125,1,12,0.16,False,True,False,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,BR-116,2094,VEN4E56,745.0,860.0,856.0,2093.0,4.0,-2.0,6.00,867.000,1,9,0.12,False,True,False,False,0.0,0
681,BR-116,2095,VEN4E56,745.0,854.0,860.0,2094.0,-6.0,4.0,-10.00,843.000,1,9,0.12,False,True,False,False,0.0,0
682,BR-116,2096,VEN4E56,745.0,848.0,854.0,2095.0,-6.0,-6.0,0.00,842.000,0,9,0.12,False,False,False,False,0.0,0
683,BR-116,2097,VEN4E56,745.0,850.0,848.0,2096.0,2.0,-6.0,8.00,856.000,1,9,0.12,False,True,False,False,0.0,0


In [36]:
data = df.select('rodovia', 'tempo_da_simulacao', 'placa', 'vel_y',F.lit(1.2))
data = data.toPandas()
data

,rodovia,tempo_da_simulacao,placa,vel_y,1.2
0,BR-116,2000,BOL5P67,NaN,1.2
1,BR-116,2001,BOL5P67,-2.00,1.2
2,BR-116,2002,BOL5P67,2.00,1.2
3,BR-116,2003,BOL5P67,0.00,1.2
4,BR-116,2005,BOL5P67,0.00,1.2
...,...,...,...,...,...
680,BR-135,2054,GUY1X23,-5.75,1.2
681,BR-135,2055,GUY1X23,-446.85,1.2
682,BR-135,2056,GUY1X23,-472.85,1.2
683,BR-135,2057,GUY1X23,NaN,1.2


In [34]:
F.lit(1.2)

Column<'1.2'>

In [15]:
data = df.select('rodovia', 'tempo_da_simulacao', 'placa', 'vel_y',F.lit(1.2))
data = data.toPandas()

query = f""" INSERT IGNORE INTO carros (placa, pos_x, pos_y, aceleracao, velocidade, rodovia, horario_registro, multas, risco_colisao, direcao_perigosa)

        VALUES {','.join([str(i) for i in list(datap.to_records(index=False))])};
                
        """.replace("None", "NULL").replace("\n", "").replace("nan", "NULL")

# cursor.execute(query)
# connection.commit()

# horario_registro => tempo simulacao
# velocidade_media => vel_media (pipeline4)
# tempo_medio_cruzamento => tempo_medio   (pipeline4)
# tempo_processamento => ??
# total_veiculos => count(placas)
# veiculos_colisao => count(vel_y == 0)


In [ ]:
CREATE TABLE rodovias (
    nome_rodovia varchar(20)  NOT NULL,
    horario_registro bigint  NOT NULL,
    velocidade_media float(6,2)  NOT NULL,
    tempo_medio_cruzamento float(6,2)  NOT NULL,
    tempo_processamento float(12,4)  NOT NULL,
    total_veiculos int  NULL,
    veiculos_acima_vel int  NOT NULL,
    veiculos_colisao int  NOT NULL,
    CONSTRAINT rodovias_pk PRIMARY KEY (horario_registro,nome_rodovia)
);

In [13]:
data = df.select('rodovia', 'tempo_da_simulacao')
data = data.toPandas()

In [32]:

data['placa'].loc[data['tempo_da_simulacao'] == max(data['tempo_da_simulacao'])].unique

data['placa'].loc[(data['tempo_da_simulacao'] == max(data['tempo_da_simulacao'])) & (data['vel_y'] == 0) ].unique


<bound method Series.unique of 356    PAR7P89
456    PER1H23
Name: placa, dtype: object>

In [17]:
max(data['tempo_da_simulacao'])

2099

In [73]:
from pyspark.sql import functions as F, Window

# Obter o valor máximo da coluna tempo_da_simulacao
df2 = df.select('rodovia', 'tempo_da_simulacao')
max_tempo = df2.select(F.max("tempo_da_simulacao")).collect()[0][0]

# Filtrar o dataframe pelo valor máximo
df2 = df2.filter(df2.tempo_da_simulacao == max_tempo)

# Aplicar um unique no dataframe df2
df2 = df2.distinct()

max_tempo = df2.select(F.max("tempo_da_simulacao")).collect()[0][0]
df2 = df2.filter(df2.tempo_da_simulacao == max_tempo)

# Criar uma janela de tamanho 1 sobre a coluna rodovia
w = Window.partitionBy(F.col("rodovia")).orderBy(F.col("rodovia")).rangeBetween(0, 0)

# Contar as placas distintas por rodovia
unique_placas = df.filter(df.tempo_da_simulacao == max_tempo_da_simulacao)\
                  .withColumn("placa_distinct", F.size(F.collect_set("placa").over(w)))\
                  .select("rodovia", "placa_distinct")\
                  .distinct()

# Contar as placas distintas por rodovia que tiveram colisão
unique_colisao = df.filter((df.tempo_da_simulacao == max_tempo_da_simulacao) & (df.vel_y == 0))\
                  .withColumn("placa_distinct_colissao", F.size(F.collect_set("placa").over(w)))\
                  .select("rodovia", "placa_distinct_colissao")\
                  .distinct()

df2 = df2.join(unique_placas,on='rodovia',how='inner')
df2 = df2.join(unique_colisao,on='rodovia',how='inner')


end_time = time.time()
elapsed_time = end_time - start_time2
elapsed_time

df2 = df2.withColumn('tempo_processamento', F.lit(elapsed_time))
df2 = df2.withColumn('tempo_processamento', F.lit(elapsed_time))
df2 = df2.withColumn('tempo_processamento', F.lit(elapsed_time))


df2.show()



+-------+------------------+--------------+-----------------------+-------------------+
|rodovia|tempo_da_simulacao|placa_distinct|placa_distinct_colissao|tempo_processamento|
+-------+------------------+--------------+-----------------------+-------------------+
| BR-116|              2099|             9|                      2|  5513.651318788528|
+-------+------------------+--------------+-----------------------+-------------------+



In [74]:
datap2 = df2.toPandas()

In [77]:
query = f""" INSERT IGNORE INTO carros (nome_rodovia,horario_registro,total_veiculos,veiculos_colisao,tempo_processamento)

        VALUES {','.join([str(i) for i in list(datap2.to_records(index=False))])};
                
        """.replace("None", "NULL").replace("\n", "").replace("nan", "NULL")

In [78]:
query

" INSERT IGNORE INTO carros (rodovia, pos_x, pos_y, aceleracao, velocidade, rodovia, horario_registro, multas, risco_colisao, direcao_perigosa)        VALUES ('BR-116', 2099, 9, 2, 5513.65131879);                        "

In [ ]:
CREATE TABLE rodovias (
    nome_rodovia varchar(20)  NOT NULL,
    horario_registro bigint  NOT NULL,
    velocidade_media float(6,2)  NULL,
    tempo_medio_cruzamento float(6,2)  NULL,
    tempo_processamento float(12,4)  NULL,
    total_veiculos int  NULL,
    veiculos_colisao int  NULL,
    CONSTRAINT rodovias_pk PRIMARY KEY (horario_registro,nome_rodovia)
);